In [34]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from collections import Counter
from torch.utils.data import TensorDataset, DataLoader

data = pd.read_csv('tweets_10000.csv')
reviews = np.array(data['tweet_text'])[:10000]
labels = np.array(data['label'])[:10000]

In [35]:
# preview data
data.head()

,label,created_at,tweet_id,tweet_text,retweet_count,favorite_count,tweet_lang,user_id,user_name,user_screen_name,user_since,user_time_zone,user_verified
0,2,Mon Jul 13 13:32:54 +0000 2020,1282669333798240257,"*Maksudnya uang dibalik meja,Bisa di katakan K...",0,0,in,1281767862131023872,SinoJardin,JardinSino,Sat Jul 11 01:51:12 +0000 2020,NaN,False
1,2,Mon Jul 13 13:32:53 +0000 2020,1282669329645879297,#BURUHTOLAKAKSI16JULI SIKAP BURUH KERAS MENYAT...,0,0,in,1224937759678193664,Ferdy Simbolon,SimbolonFerdy,Wed Feb 05 06:08:40 +0000 2020,NaN,False
2,0,Mon Jul 13 13:32:51 +0000 2020,1282669320779124736,Ahh sama je la semuanya bodo baik muda ke tua ...,0,0,in,353955052,ᵃˢ 🇲🇾,apik1323,Fri Aug 12 22:20:51 +0000 2011,NaN,False
3,0,Mon Jul 13 13:32:51 +0000 2020,1282669319445286914,#BURUHTOLAKAKSI16JULI SIKAP BURUH KERAS MENYAT...,0,0,in,1268840238765404160,Nikita fris,fris_niki,Fri Jun 05 09:41:14 +0000 2020,NaN,False
4,2,Mon Jul 13 13:32:47 +0000 2020,1282669304714940416,#BURUHTOLAKAKSI16JULI SIKAP BURUH KERAS MENYAT...,0,0,in,1207860942240804865,Randi Radika,radika_randi,Fri Dec 20 03:11:15 +0000 2019,NaN,False


In [36]:
# preview counter
# 1 = Positif; 2 = Negatif; 0 = Netral
Counter(labels)

Counter({2: 3369, 0: 3378, 1: 3253})

In [37]:
# functions

# padding sequences
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

# tokenizer
def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()
    
    # get rid of web address, twitter id, and digit
    new_text = []
    for word in test_words:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in new_text])

    return test_ints

# prediction
def predict(net, test_review, sequence_length=30):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positif.")
    elif(pred.item()==0):
        print("Netral.")
    else:
        print("Negatif.")

In [38]:
# classes

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [39]:
# pre-processing
punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'

# get rid of punctuation
all_reviews = 'separator'.join(reviews)
all_reviews = all_reviews.lower()
all_text = ''.join([c for c in all_reviews if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('separator')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()
# get rid of web address, twitter id, and digit

new_reviews = []
for review in reviews_split:
    review = review.split()
    new_text = []
    for word in review:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    new_reviews.append(new_text)
    
## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in new_reviews:
    reviews_ints.append([vocab_to_int[word] for word in review])
    
# 1 = positif, 2 = negatif, 0 = netral label conversion
encoded_labels = []
for label in labels:
    if label == 1:
        encoded_labels.append(1)
    elif label == 0:
        encoded_labels.append(0)
    else:
        encoded_labels.append(2)

encoded_labels = np.asarray(encoded_labels)

# test implementation!
seq_length = 30
features = pad_features(reviews_ints, seq_length=seq_length)

## test statements 
assert len(features)==len(reviews_ints), "The features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

## split data into training, validation, and test data (features and labels, x and y)
split_frac = 0.8
split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of the resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))



# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE the training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

			Feature Shapes:
Train set: 		(8000, 30) 
Validation set: 	(1000, 30) 
Test set: 		(1000, 30)


In [40]:
# preparing network

# first checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')
    
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 200
hidden_dim = 128
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

No GPU available, training on CPU.


In [41]:
# training

# loss and optimization functions
lr=0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

epochs = 10 
counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/10... Step: 100... Loss: -1.233672... Val Loss: -0.010508
Epoch: 2/10... Step: 200... Loss: 0.266463... Val Loss: -0.011516
Epoch: 2/10... Step: 300... Loss: 0.082972... Val Loss: -0.007509
Epoch: 3/10... Step: 400... Loss: -1.080732... Val Loss: 0.032306
Epoch: 4/10... Step: 500... Loss: 0.195846... Val Loss: 0.196909
Epoch: 4/10... Step: 600... Loss: -0.722359... Val Loss: -0.260807
Epoch: 5/10... Step: 700... Loss: -22.557310... Val Loss: -0.325737
Epoch: 5/10... Step: 800... Loss: -24.657799... Val Loss: 0.469900
Epoch: 6/10... Step: 900... Loss: -18.267254... Val Loss: -0.028774
Epoch: 7/10... Step: 1000... Loss: -31.811924... Val Loss: -0.307216
Epoch: 7/10... Step: 1100... Loss: -25.266123... Val Loss: -0.037911
Epoch: 8/10... Step: 1200... Loss: -28.250751... Val Loss: -0.320075
Epoch: 9/10... Step: 1300... Loss: -31.906994... Val Loss: 0.987543
Epoch: 9/10... Step: 1400... Loss: -39.721298... Val Loss: -0.428554
Epoch: 10/10... Step: 1500... Loss: -18.023981... Val Lo

In [42]:
# testing

# get test data loss and accuracy
test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: -1.489
Test accuracy: 0.349


In [43]:
# predicting

# negative test review
test_review = "jangan jadikan politik sebagai alat bung"
# test code and generate tokenized review
test_ints = tokenize_review(test_review)
# test sequence padding
seq_length=30
features = pad_features(test_ints, seq_length)
# test conversion to tensor and pass into your model
feature_tensor = torch.from_numpy(features)
seq_length = 30 # good to use the length that was trained on

In [47]:
# call function on negative review
test_review_neg = "@Bunnga8 Ini politik adu domba, eh bukan deng. Pokonya itulah istilahnya"
predict(net, test_review_neg, seq_length)

Prediction value, pre-rounding: 1.000000
Positif.


In [48]:
# call function on positive review
test_review_pos = "@collegemenfess Hmm semua akan tersasa tantangannya kalo udah bahas politik"
predict(net, test_review_pos, seq_length)

Prediction value, pre-rounding: 1.000000
Positif.


In [49]:
# call function on neutral review
test_review_neu = "Tahun Pelajaran 2020/2021, Madrasah Gunakan Kurikulum PAI Baru"
predict(net, test_review_neu, seq_length)

Prediction value, pre-rounding: 0.084279
Netral.
